In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os

print("GRO620 - Problématique")
print("OpenCV version", cv.__version__)

%matplotlib inline

In [ ]:
# Load images

images_fn = os.listdir("photos_prob/")
images_fn.sort()
print("%i photo(s) à traiter" % (len(images_fn)))
if len(images_fn) == 0:
    print(
        "ERREUR! Vérifiez que vous avez bien un dossier photos_prob au même endroit que ce calepin."
    )

images: list[cv.typing.MatLike] = []

for f in images_fn:
    img_path = os.path.join("photos_prob/", f)
    img = cv.imread(img_path)
    assert img is not None
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    images.append(img)

plt.figure(0)
for index, image in enumerate(images):
    plt.subplot(3, 3, index + 1)
    plt.imshow(image)
    plt.title(f"Original {index + 1}")
    plt.axis("off")
plt.show()


# Paramètres Intrinsèques

Pour déterminer les valeurs $f_x$ et $f_y$ de la matrice $K$, on utilise les équivalences suivantes:

$$ \frac{f}{f_x} = \frac{largeur_{capteur}}{largeur_{image}} $$
$$ \frac{f}{f_y} = \frac{hauteur_{capteur}}{hauteur_{image}} $$


In [ ]:
# 1. Parametres de la camera

FOCAL_LENGTH = 23.0e-3  # m
IMG_WIDTH = 640  # px
IMG_HEIGHT = 427  # px
SENSOR_WIDTH = 23.4e-3  # m
SENSOR_HEIGHT = 15.6e-3  # m

# Calcul des parametres intrinseques
fx = FOCAL_LENGTH * (IMG_WIDTH / SENSOR_WIDTH)  # px
fy = FOCAL_LENGTH * (IMG_HEIGHT / SENSOR_HEIGHT)  # px
cx = IMG_WIDTH / 2.0  # px
cy = IMG_HEIGHT / 2.0  # px
skew = 0.0  # px

K_tilde = np.array(
    [
        [fx, skew, cx, 0],
        [0, fy, cy, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

print(f"Matrice intrinseque K:\n{K_tilde}")


# Matrices de transformation et de projection

À partir de l'équation 2.64 du manuel de cours, on dérive l'équation pour calculer la matrice de projection en coordonnées homogènes:

$$ \tilde{P} = \tilde{T}_{c}\cdot\tilde{K}^{-1}, $$

où $\tilde{T}_{c}$ est la matrice de transformation en coordonnées homogènes du repère $\{C\}$ au repère $\{0\}$.

Puisque le plan $XY$ du repère $\{0\}$ est coplanaire avec le convoyeur, on dérive la distance verticale entre le convoyeur et la caméra:

$$ z_{c} \equiv z_{s} = \tilde{T}_{c, \{3, 4\}} $$


In [ ]:
# 2. Matrice de transformation extrinseque w_T_c (repere {C} au repere {0})

w_T_c_tilde = np.array(
    [
        [1, 0, 0, 0.500],
        [0, -1, 0, 0.200],
        [0, 0, -1, 0.282],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

print(f"Matrice extrinseque w_T_c (Camera -> World):\n{w_T_c_tilde}\n")

# Matrice de projection complete

w_P_c_tilde = w_T_c_tilde @ np.linalg.inv(K_tilde)

print(f"Matrice de projection complete (Camera -> World):\n{w_P_c_tilde}\n")

# Calcul de z_c
X_0 = 0  # m
Y_0 = 0  # m
Z_0 = 0  # m
z_conveyor = (
    w_T_c_tilde[2, 3]
    + w_T_c_tilde[2, 0] * X_0
    + w_T_c_tilde[2, 1] * Y_0
    + w_T_c_tilde[2, 3] * Z_0
)  # m

print(f"z_c: {z_conveyor:.3f} m")


# Traitement de l'image et extraction des caractéristiques

Dans les images à traiter, on observe certaines particularités:

- Les objets à identifier possèdent une couleur distincte du reste de l'image;
- Les objets à identifier possèdent une forme similaire;
- L'arrière-plan (convoyeur) n'est pas uniforme, et représente du bruit.

On cherche à identifier et caractériser les vis dans les images. On sépare le problème en deux volets:

1. Isoler les vis dans l'image;
1. Déterminer la pose des vis dans l'image.

L'isolation des vis dans les images se fait par une chaîne de traitement d'image. Les paramètres de chaque étape de la chaîne sont déterminés de façon expérimentale (*i.e.I essai-erreur). La chaîne d'acquisition proposée implémente la logique suivante:

1. Réduire le bruit de l'arrière-plan, de sorte à l'uniformiser;
1. Appliquer un masque binaire à l'image afin de ségréger les vis et l'arrière-plan;
1. Appliquer des opérations morphologiques afin d'éliminer le bruit résiduel, et assurer que la forme originale des vis soit préservée (nécessaire pour une détection de contours idéale).

Pour déterminer la pose des vis dans l'image, une seconde chaîne de traitement implémente la logique suivante:

1. Détecter les contours des vis;
1. Filter les contours résultant du bruit résiduel;
1. Trouver la boîte englobante de chaque vis afin d'extraire leurs caractéristiques de position, de dimensions, et d'orientation;
1. Standardiser la notation des caractéristiques des vis;
1. Classifier les vis en fonction de leurs dimensions;
1. Convertir la position 2D dans l'image en coordonnées 3D.

Additionnellement, on dessine les boîtes englobantes (accompagnées de leur identifiant respectif) sur l'image originale afin de vérifier visuellement les résultats.

L'équation de conversion en coordonnées 3D est dérivée de l'équation 2.67, ainsi que des équations du document "Transformation 2D à 3D" fourni dans le cadre de la problématique.

## Note sur le calcul de l'orientation

Le calcul de l'orientation des vis est effectué à partir de la boîte englobante. Toutefois, tel qu'observé dans les images annotées, l'orientation des boîtes diffère un peu de l'orientation des vis: il y a donc une certaine tolérance associée aux valeurs de l'orientation (de l'ordre de quelques degrés).

Une stratégie qui résulterait en des valeurs plus justes serait d'utiliser la transformée de Hough pour identifier les lignes "directrices" dans l'image, qui sont parallèles aux vis. Cette stratégie possède aussi des limites: plusieurs lignes peuvent appartenir à la même vis. Les résultats de la transformée nécessite donc d'être agglomérer (*clustering*). Tel qu'observé dans le code ci-dessous, les paramètres de l'algorithme d'agglomération qui permettent d'obtenir un ratio $1:1$ (c.à.d. une ligne par vis) n'ont pas été identifiés. Considérant que l'agglomération des lignes est en dehors de la portée du cours, les résultats produits par l'algorithme d'agglomération ne sont pas utilisés.


In [ ]:
from sklearn.cluster._hdbscan.hdbscan import HDBSCAN

processed = []

all_results = []

for index, image in enumerate(images):
    original = image.copy()
    blurred = cv.GaussianBlur(original, (7, 7), 3.0)
    gray = cv.cvtColor(blurred, cv.COLOR_RGB2GRAY)
    _, threshold = cv.threshold(gray, 180, 255, cv.THRESH_BINARY)
    kernel = np.ones((3, 3), dtype=np.uint8)
    opening = cv.morphologyEx(threshold, cv.MORPH_OPEN, kernel)
    dilated = cv.dilate(opening, kernel, iterations=1)
    processed.append(dilated)

    lines = cv.HoughLinesP(
        dilated, 2.0, np.pi / 180, 100, minLineLength=25, maxLineGap=5
    )

    # Cluster lines detected by Hough transform
    # NOTE: ideally, we want one line per screw, but this is rather difficult to obtain with clustering.
    midpoints = np.array(
        [
            [(line[0][0] + line[0][2]) / 2, (line[0][1] + line[0][3]) / 2]  # type: ignore
            for line in lines
        ]
    )
    clusterer = HDBSCAN(
        min_cluster_size=2
    )  # Large-ish `n` samples, medium `n` clusters, cluster by distance betweem nearest points
    labels = clusterer.fit_predict(midpoints)
    # Average lines for each cluster label
    clustered_lines = []
    for lbl in np.unique(labels):
        if lbl == -1:
            continue  # skip noise
        idxs = np.where(labels == lbl)[0]
        # Average the endpoints of all lines in this cluster
        avg_line = np.mean([lines[i][0] for i in idxs], axis=0)
        clustered_lines.append(avg_line.astype(int))
    lines = np.array(clustered_lines).reshape(-1, 1, 4)

    # print(f"Image {index + 1}: {len(lines)} lines detected")

    for line in lines:
        x1, y1, x2, y2 = line[0]
        theta = np.rad2deg(np.atan2(y2 - y1, x2 - x1))
        cv.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Identify, classify, and compute pose
    contours, _ = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    MIN_SCREW_AREA = 100  # px
    MAX_SCREW_AREA = 2000  # px
    HEIGHT_THRESHOLD = 100.0  # px

    # Extract caracterstics
    results = []
    screw_id: int = 0
    for contour in contours:
        area = cv.contourArea(contour)
        if area < MIN_SCREW_AREA or area > MAX_SCREW_AREA:
            continue

        rect = cv.minAreaRect(contour)
        (c_u, c_v), (width, height), theta = rect

        # Draw bounding box and id
        bbox = cv.boxPoints(rect)
        bbox = np.intp(bbox)  # Convert box points to integer type for drawing
        cv.drawContours(image, [bbox], 0, (0, 0, 255), 2)  # type: ignore
        cv.putText(
            image,
            str(screw_id),
            (int(c_u) - 5, int(c_v) - 5),
            cv.FONT_HERSHEY_SIMPLEX,
            1.0,
            (255, 0, 0),
            3,
            cv.LINE_AA,
        )

        # Standardise notation
        if width > height:
            width, height = height, width
            theta += 90
        theta = (theta + 90) % 180
        theta = (360 - theta) % 180

        screw_type = "courte"
        if height > HEIGHT_THRESHOLD:
            screw_type = "longue"

        # # px -> m ({C} frame)
        z_screen = z_conveyor  # px?
        x_screen = c_u * z_screen  # px^2?
        y_screen = c_v * z_screen  # px^2?
        X_s_prime = np.array([x_screen, y_screen, z_screen, 1], dtype=np.float32)

        # # {C} -> {0}
        p_w = w_P_c_tilde @ X_s_prime
        x_0, y_0, z_0, _ = p_w

        results.append(
            {
                "id": screw_id,
                "type": screw_type,
                "X": x_0,
                "Y": y_0,
                "Z": z_0,
                "theta": theta,
            }
        )
        screw_id += 1

    all_results.append(results)

plt.figure(0)
for index, image in enumerate(processed):
    plt.subplot(3, 3, index + 1)
    plt.imshow(image, "gray")
    plt.title(f"Image {index + 1}")
    plt.axis("off")

plt.show()


# Analyse des résultats

Les faiblesses suivantes de la chaîne de traitement ont été identifiées:

- L'orientation des vis est indépendante de la tête, et la précision n'est pas idéale (la tolérance est de l'ordre de $\pm 10\degree$);
- La chaîne de traitement n'a pas été évaluée dans une situation différente. Une image où des débris s'entremêlent avec les vis pourrait résulter en une performance médiocre. Autrement dit, la chaîne de traitement n'est pas robuste.

Un moyen de mitiger l'erreur sur l'orientation des vis serait d'augmenter l'ouverture du préhenseur du robot manipulateur afin d'assurer le succès d'une préhension sur l'entièreté de la plage de valeurs possibles.

La force principale de la chaîne de traitement identifiée est: "ça fonctionne!".

Considérant que seulement une vis n'a pas été détectée (voir 'image 5'), et que la majorité de cette vis était à l'extérieur du champ de vision de la caméra (donc manque d'information pour extraire ses caractéristiques), et considérant que quelques boîtes englobantes n'englobent pas complètement leur vis respective, mais que l'algorithme de classification identifie correctement le type de la vis, et que l'erreur en position est de l'ordre de quelques pixels, la performance de la chaîne de traitement est jugé suffisante pour répondre aux besoins du problème posé.


In [ ]:
# Print results

import tabulate

for index, _ in enumerate(images):
    print(f"Image {index + 1}:\n")
    print(
        tabulate.tabulate(
            [
                [
                    screw["id"],
                    screw["type"],
                    screw["X"],
                    screw["Y"],
                    screw["Z"],
                    screw["theta"],
                ]
                for screw in all_results[index]
            ],
            headers=[
                "id",
                "Type",
                "X (m)",
                "Y (m)",
                "Z (m)",
                "theta (deg)",
            ],
            floatfmt=".3f",
        )
    )
    print()

plt.figure()
for index, image in enumerate(images):
    plt.subplot(3, 3, index + 1)
    plt.imshow(image)
    plt.title(f"Image {index + 1}")
    plt.axis("off")
plt.show()
